In [1]:
%pip install transformers


Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

c:\Users\Sheikh\anaconda3\envs\fyp_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Sheikh\anaconda3\envs\fyp_env\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\Sheikh\anaconda3\envs\fyp_env\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
cannot import name 'split_torch_state_dict_into_shards' from 'huggingface_hub' (c:\Users\Sheikh\anaconda3\envs\fyp_env\Lib\site-packages\huggingface_hub\__init__.py)

In [3]:
import torch
print(dir(torch))


['__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'utils']


In [4]:
df = pd.read_csv("Reddit_Data.csv")
df = df.rename(columns={'clean_comment': 'content', 'category': 'sentiment'})
df.head()


,content,sentiment
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [5]:
label_map = {-1: 0, 0: 1, 1: 2}  # negative, neutral, positive mapping
df['label'] = df['sentiment'].map(label_map)
df = df.dropna(subset=['label'])  # remove rows with unmapped sentiment
df['label'] = df['label'].astype(int)
df.head()


,content,sentiment,label
0,family mormon have never tried explain them t...,1,2
1,buddhism has very much lot compatible with chr...,1,2
2,seriously don say thing first all they won get...,-1,0
3,what you have learned yours and only yours wha...,0,1
4,for your own benefit you may want read living ...,1,2


In [6]:
df = df.dropna(subset=['content'])  # drop rows with empty content
df['content'] = df['content'].astype(str)  # ensure content is string
df.head()


,content,sentiment,label
0,family mormon have never tried explain them t...,1,2
1,buddhism has very much lot compatible with chr...,1,2
2,seriously don say thing first all they won get...,-1,0
3,what you have learned yours and only yours wha...,0,1
4,for your own benefit you may want read living ...,1,2


# Splitting data into train test splits

In [7]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['content'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)

print(f"Training samples: {len(train_texts)}")
print(f"Validation samples: {len(val_texts)}")


Training samples: 29719
Validation samples: 7430


# BERT Tokenizer to convert words into vectors

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
# Tokenize the text data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)


# Dataloader

In [17]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.reset_index(drop=True)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Cast label to long tensor to avoid runtime error
        item['labels'] = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        return item
    
    def __len__(self):
        return len(self.labels)


In [18]:
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)


# Bert model for sentiment classification

In [19]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', 
    num_labels=3  # since you have 3 sentiment classes: negative, neutral, positive
)

loading configuration file config.json from cache at C:\Users\Sheikh\.cache\huggingface\hub\models--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.52.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file model.safetensors from 

In [22]:
def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    p, r, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': p,
        'recall': r
    }

# Training args

In [24]:
%pip install transformers[torch]
%pip install accelerate>=0.26.0


   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.5/204.2 MB 16.4 MB/s eta 0:00:13
   ---------------------------------------- 1.6/204.2 MB 6.0 MB/s eta 0:00:34
    --------------------------------------- 3.7/204.2 MB 7.5 MB/s eta 0:00:27
   - -------------------------------------- 5.8/204.2 MB 8.4 MB/s eta 0:00:24
   - -------------------------------------- 8.1/204.2 MB 9.0 MB/s eta 0:00:22
   -- ------------------------------------- 10.7/204.2 MB 9.6 MB/s eta 0:00:21
   -- ------------------------------------- 13.4/204.2 MB 10.1 MB/s eta 0:00:19
   --- ------------------------------------ 16.5/204.2 MB 10.7 MB/s eta 0:00:18
   --- ------------------------------------ 19.7/204.2 MB 11.3 MB/s eta 0:00:17
   ---- ----------------------------------- 23.3/204.2 MB 11.9 MB/s eta 0:00:16
   ----- ---------------------------------- 27.5/204.2 MB 12.6 MB/s eta 0:00:14
   ------ --------------------------------- 32.0/204.2 MB 1

  You can safely remove it manually.


Note: you may need to restart the kernel to use updated packages.


In [23]:
# training_args = TrainingArguments(
#     output_dir='./results',          # output directory
#     num_train_epochs=50,              # total number of training epochs
#     per_device_train_batch_size=64,  # batch size per device during training
#     per_device_eval_batch_size=64,   # batch size for evaluation
#     warmup_steps = 500,
#     weight_decay=0.01,               # strength of weight decay
#     logging_dir='./logs',            # directory for storing logs
#     logging_steps=10,
#     evaluation_strategy='epoch',
#     save_strategy='epoch',
#     log_level='info',
#     fp16=True,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     compute_metrics=compute_metrics,
# )

from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',            # output directory to save model checkpoints and logs
    num_train_epochs=2,                # number of training epochs
    per_device_train_batch_size=64,   # batch size for training
    per_device_eval_batch_size=64,    # batch size for evaluation
    warmup_steps=500,                  # warmup steps for learning rate scheduler
    weight_decay=0.01,                # weight decay to regularize training
    logging_dir='./logs',              # directory to store training logs
    logging_steps=10,                  # log training info every 10 steps
    eval_strategy='epoch',             # evaluate model at end of each epoch
    save_strategy='epoch',             # save model at end of each epoch
    log_level='info',
    fp16=True,                        # mixed precision training for speedup (if supported)
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using auto half precision backend


In [ ]:
trainer.train()

In [1]:
import torch
print(torch.cuda.is_available())      # Should print True
print(torch.cuda.get_device_name(0)) # Should print your GPU model


False


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

In [ ]:
log_history = trainer.state.log_history

training_losses = [log['loss'] for log in log_history if 'loss' in log]
evaluation_losses = [log['eval_loss'] for log in log_history if 'eval_loss' in log]

plt.figure(figsize=(10, 5))
plt.plot(training_losses, label='Training Loss')
plt.plot(evaluation_losses, label='Evaluation Loss')
plt.xlabel('Logging Steps')
plt.ylabel('Loss')
plt.title('Training and Evaluation Loss Over Time')
plt.legend()
plt.show()

In [ ]:
trainer.save_model('sentiment-model')

In [ ]:
tokenizer.save_pretrained('sentiment-model', safe_serialization=True)

In [ ]:
%pip install streamlit
%pip install captum streamlit transformers


In [ ]:
#ss

In [15]:
def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    p, r, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': p,
        'recall': r
    }


In [ ]:
# Train the model
##trainer.train()

# Plot training & validation accuracy
train_acc = []
eval_acc = []
train_f1 = []
eval_f1 = []
train_precision = []
eval_precision = []
train_recall = []
eval_recall = []

# Loop through the log history and extract the metrics
for log in trainer.state.log_history:
    if 'eval_accuracy' in log:
        eval_acc.append(log['eval_accuracy'])
        eval_f1.append(log['eval_f1'])
        eval_precision.append(log['eval_precision'])
        eval_recall.append(log['eval_recall'])
    if 'train_accuracy' in log:
        train_acc.append(log['train_accuracy'])
        train_f1.append(log['train_f1'])
        train_precision.append(log['train_precision'])
        train_recall.append(log['train_recall'])

# Plot accuracy
plt.figure(figsize=(12, 6))
plt.plot(train_acc, label="Train Accuracy")
plt.plot(eval_acc, label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Train vs Validation Accuracy")
plt.show()

# Plot precision
plt.figure(figsize=(12, 6))
plt.plot(train_precision, label="Train Precision")
plt.plot(eval_precision, label="Validation Precision")
plt.xlabel("Epochs")
plt.ylabel("Precision")
plt.legend()
plt.title("Train vs Validation Precision")
plt.show()

# Plot F1 score
plt.figure(figsize=(12, 6))
plt.plot(train_f1, label="Train F1 Score")
plt.plot(eval_f1, label="Validation F1 Score")
plt.xlabel("Epochs")
plt.ylabel("F1 Score")
plt.legend()
plt.title("Train vs Validation F1 Score")
plt.show()

# Plot recall
plt.figure(figsize=(12, 6))
plt.plot(train_recall, label="Train Recall")
plt.plot(eval_recall, label="Validation Recall")
plt.xlabel("Epochs")
plt.ylabel("Recall")
plt.legend()
plt.title("Train vs Validation Recall")
plt.show()
